# Station Visibility

In [ ]:
from fds.config import set_api_key, set_url
set_url('https://api.spacetower.exotrail.space/fds/v1')
set_api_key('MY_API_KEY')

In [ ]:
from datetime import datetime, timedelta, UTC
import pandas as pd
from fds.models.ground_station import GroundStation
from fds.models.orbit_extrapolation.requests import EventsRequestStationVisibility
import plotly.express as px

from fds.models.orbit_extrapolation.use_case import OrbitExtrapolation
from fds.models.orbital_state import OrbitalState, PropagationContext
from fds.models.spacecraft import SpacecraftSphere
from fds.models.two_line_element import TwoLineElement

In [ ]:
initial_tle = TwoLineElement(
    "1 58295U 23174AR  24137.17541737  .00005775  00000-0  29855-3 0  9994",
    "2 58295  97.4580 211.9750 0010360 352.3995   7.7079 15.16607347 28836"
)

In [ ]:
propagation_context = PropagationContext(
    model_perturbations=[
        PropagationContext.Perturbation.DRAG,
        PropagationContext.Perturbation.EARTH_POTENTIAL,
        PropagationContext.Perturbation.SRP,
        PropagationContext.Perturbation.THIRD_BODY,
    ],
    model_solar_flux=150,  # SFU
    model_earth_potential_deg=30,
    model_earth_potential_ord=30,
    model_atmosphere_kind=PropagationContext.AtmosphereModel.HARRIS_PRIESTER,
    integrator_kind=PropagationContext.IntegratorKind.DORMAND_PRINCE_853,
    integrator_min_step=0.01,  # s
    integrator_max_step=100,  # s
)

spacecraft = SpacecraftSphere(
    platform_mass=150,  # kg
    drag_coefficient=2.2,
    cross_section=10,
    reflectivity_coefficient=0.6
)

orbital_state = OrbitalState.from_tle(
    tle=initial_tle,
    propagation_context=propagation_context,
    spacecraft=spacecraft,
)

target_date = datetime.now(UTC) + timedelta(
    seconds=orbital_state.mean_orbit.keplerian_period * 3)

station_events_request = EventsRequestStationVisibility(
    start_date=orbital_state.date,
    ground_stations=[GroundStation("Toulouse", 43.6047, 1.4442, 0.115, 5, )]
)

In [ ]:
oe = OrbitExtrapolation.with_target_date(
    target_date=target_date,
    initial_orbital_state=orbital_state,
    station_visibility_events_request=station_events_request,
)

res = oe.run().result

In [ ]:
df_timeline = pd.DataFrame(res.export_event_gantt_data())
df_timeline = df_timeline[df_timeline["event"] != "DESCENDING_NODE"]
df_timeline = df_timeline[df_timeline["event"] != "ASCENDING_NODE"]
fig = px.timeline(df_timeline, x_start="start_date", x_end="end_date", y="event", color="event",
                  labels={'Task': 'Event'},
                  title='Events timeline',
                  hover_name="event",
                  hover_data={"ground_station_name": True}
                  )
fig.show()